In [ ]:
import sys
sys.path.append('../helper')
from ls_helper import *

In [ ]:
from label_studio_sdk.label_interface import LabelInterface
from label_studio_sdk.label_interface.create import labels
from label_studio_sdk.actions import ActionsCreateRequestSelectedItemsExcluded
from dotenv import load_dotenv
import os
import logging

In [ ]:
load_dotenv()

In [ ]:
logging.basicConfig( level=logging.DEBUG)

In [ ]:
logging.info('Setting up required variable values')

# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL', 'http://localhost:8080')
LABEL_STUDIO_ML_BACKEND_URL = os.getenv('LABEL_STUDIO_ML_BACKEND_URL', 'http://localhost:9090')

# API key is available at the Account & Settings > Access Tokens page in Label Studio UI
API_KEY = os.getenv('API_KEY')

# Import the SDK and the client module
from label_studio_sdk.client import LabelStudio

# Connect to the Label Studio API and check the connection
ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)

In [ ]:
FLAIR_PROJECT_NAME = os.getenv('FLAIR_PROJECT_NAME', 'test-flair-ner')
RECREATE_PROJECT = int(os.getenv('RECREATE_PROJECT', '0'))
LOAD_TASKS_FROM_LOCAL_FILE = int(os.getenv('LOAD_TASKS_FROM_LOCAL_FILE', '0'))
LOCAL_TASKS_FILE='sample_data/flair_ner.txt'

# Define labeling interface
label_config = LabelInterface.create({
    'text': 'Text',
    'label': labels(['PER', 'ORG', 'LOC', 'MISC'])
})

In [ ]:
logging.info('Creating project')
# Create a project with the specified title and labeling configuration
project = get_or_create_project(ls, FLAIR_PROJECT_NAME, label_config, RECREATE_PROJECT)
logging.info('Project:' + str(project))

In [ ]:
if RECREATE_PROJECT:
    if LOAD_TASKS_FROM_LOCAL_FILE:
        logging.info('Attempting to load tasks from file')
        with open(LOCAL_TASKS_FILE, 'r') as file:
            for line in file:
                ls.projects.import_tasks(
                    id=project.id,
                    request=[
                        {"text": line.strip()}
                    ]
                )
    else:
        # Create some sample tasks
        logging.info('Creating hard-coded sample tasks')
        ls.projects.import_tasks(
            id=project.id,
            request=[
                {"text": "Copenhagen Denmark"},
                {"text": "Washington United States"},
                {"text": "Paris France"},
            ]
        )

In [ ]:
# Create and connect the ML Backend with the project
logging.info('Creating Flair ML backend')
flair_model = get_or_create_model(
    ls,
    title='FLAIR_NER',
    description='A model to perform NER using Flair',
    url=LABEL_STUDIO_ML_BACKEND_URL,
    project_id=project.id,
    is_interactive=False
)

In [ ]:
# Create annotations using ML backend for all the imported tasks 
logging.info('Attempting to get predictions for tasks')
ls.actions.create(
            id="retrieve_tasks_predictions",
            project=project.id,
            selected_items=ActionsCreateRequestSelectedItemsExcluded(
                all_=True
            )
        )

In [ ]:
# Verify the annotations predicted by the ML backend and examine confidence score
for task in ls.tasks.list(project=project.id):
    logging.info(task)

In [ ]:
response = export_annotated_data(project.id, LABEL_STUDIO_URL, API_KEY)
logging.info(response.json())

In [ ]:
logging.info('Exporting to file')
export_to_file(str(response.json()))